# Testing for E1 - COGNITION 7Subjects
fistly define the UUID of the experiment to use

In [1]:
experimentId="f7a9e88f-5e0f-4465-a5ff-1af9e2f93866"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load cognition

In [3]:
samplerate=200  #Should rethink this
inputDimension=601

In [4]:
signals, annotations, signalsMetadata = loadCOGNITIONSpindles(cognipath, returnSignals=True)

In [5]:
annotations.head()

,subjectId,channel,startTime,stopTime,duration,phase,labelerId,type,samplerate,startInd,stopInd
0,0001,C3,1938.132297,1939.202326,1.070029,2,0001,spindle,200,387626,387840
1,0001,C3,2047.217898,2048.112834,0.894936,2,0001,spindle,200,409444,409623
2,0001,C3,2170.856032,2171.634237,0.778205,3,0001,spindle,200,434171,434327
3,0001,C3,2219.766538,2220.894943,1.128405,3,0001,spindle,200,443953,444179
4,0001,C3,2263.482492,2264.357974,0.875482,3,0001,spindle,200,452696,452872


In [6]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,isOriginalSamplerate,database
0,COG001_canal_C3.txt,C3-M2,0001,39600,200,False,COGNITION
1,COG002_canal_C3.txt,C3-M2,0002,39600,200,False,COGNITION
2,COG003_canal_C3.txt,C3-M2,0003,39600,200,False,COGNITION
3,COG004_canal_C3.txt,C3-M2,0004,39600,200,False,COGNITION
4,COG005_canal_C3.txt,C3-M2,0005,39600,200,False,COGNITION


## Preprocess

broadband in 0-40Hz, previous experiments (up tp 50Hz, are now in folders as .../features_old/...)

In [7]:
def preprocessVector(vector,samplerate):
    #1. Lowpass 40Hz
    vector=filterBand(vector,[0,40],samplerate,filterOrder=4)
    #2. Z-score
    m=np.mean(vector)
    s=np.std(vector)
    vector=(vector-m)/s
    return vector

In [8]:
for subject,signal in signals.items():
    signals[subject]=preprocessVector(signal,samplerate)

In [9]:
paddedSignals={}
for index, row in signalsMetadata.iterrows():
    paddedSignals[row.subjectId]=padVectorBothSides(signals[row.subjectId],window2half(inputDimension),method='closest')

## Load experiment results

In [10]:
experimentModels = loadExperiment(experimentId,datapath)

In [11]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,ca4abebe-7534-4387-9f7e-7e56aeb102df
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ec345db7-d0e7-447a-a219-52693d52bf41
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,5efd55af-ba1f-4dd8-905e-9279a3c25d07
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,79b2270a-cfb8-4f96-987a-040fa3dc0386
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,b1cb5f16-0c62-4dd7-b5d9-16fae63b28c5
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,28b0a080-fdd1-4b49-97d4-f98a8671c508
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,acc1aab9-4594-40ce-8d0c-f2111f7a6c69


## Hyperparameter definition
this should come from a previous evaluation notebook

In [16]:
hyperThres=0.4
hyperClose=0.1
hyperDuration=0.3

## Testing with E1 criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [13]:
experimentModels=experimentModels[experimentModels.criteriumName=='E1'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId
0,0001,E1,[0001],"[0004, 0005, 0006, 0007]","[0003, 0008]",0002,ca4abebe-7534-4387-9f7e-7e56aeb102df
1,0001,E1,[0001],"[0002, 0005, 0006, 0008]","[0007, 0004]",0003,ec345db7-d0e7-447a-a219-52693d52bf41
2,0001,E1,[0001],"[0002, 0003, 0005, 0007]","[0006, 0008]",0004,5efd55af-ba1f-4dd8-905e-9279a3c25d07
3,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0006, 0004]",0005,79b2270a-cfb8-4f96-987a-040fa3dc0386
4,0001,E1,[0001],"[0002, 0003, 0007, 0008]","[0004, 0005]",0006,b1cb5f16-0c62-4dd7-b5d9-16fae63b28c5
5,0001,E1,[0001],"[0002, 0004, 0005, 0008]","[0003, 0006]",0007,28b0a080-fdd1-4b49-97d4-f98a8671c508
6,0001,E1,[0001],"[0002, 0005, 0006, 0007]","[0003, 0004]",0008,acc1aab9-4594-40ce-8d0c-f2111f7a6c69


In [18]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadDL(row.modelId,experimentId,datapath)

    testSubjectId=row.test
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load labels
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    #Predict
    inferred=model.predict(paddedSignals[testSubjectId][np.newaxis,...,np.newaxis],verbose=0).flatten()
    rawLabels=inferred>=hyperThres
    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    #processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-09-01 14:27:40         5064
metadata.json                                  2023-09-01 14:27:40           64
variables.h5                                   2023-09-01 14:27:40      1800912


Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\conv1d_4
......vars
.........0
.........1
...layers\conv1d_5
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
*************************
2 of 7
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    20

In [19]:
dumpPickle('experimentModelsTest_E1_temp.pkl',experimentModels)

In [20]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'rawF1', 'rawPrecision', 'rawRecall', 'f1', 'precision',
       'recall', 'eventF1', 'eventPrecision', 'eventRecall'],
      dtype='object')

In [21]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [22]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [23]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [24]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [25]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.687075 NaN  0.687075  0.687075  0.687075            1.0   
0003     1.0  0.844227 NaN  0.844227  0.844227  0.844227            1.0   
0004     1.0  0.764513 NaN  0.764513  0.764513  0.764513            1.0   
0005     1.0  0.629393 NaN  0.629393  0.629393  0.629393            1.0   
0006     1.0  0.644781 NaN  0.644781  0.644781  0.644781            1.0   
0007     1.0  0.697736 NaN  0.697736  0.697736  0.697736            1.0   
0008     1.0  0.589474 NaN  0.589474  0.589474  0.589474            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.756219 NaN  0.756219  0.756219  0.756219         1.0  0.629167 NaN   
0003  0.879442 NaN  0.879442  0.879442  0.879442         1.0  0.811543 NaN   
0004  0.708502 NaN  0.708502  0.708502  0.708502         1.0  0.830549 NaN   
0005  0.476726 NaN  0.476726  0.476726  0.476726         1.0  0.930380 NaN   
0006  0.521739 NaN  0.521739  0.521739  0.521739         1.0  0.845133 NaN   
0007  0.691293 NaN  0.691293  0.691293  0.691293         1.0  0.704301 NaN   
0008  0.583333 NaN  0.583333  0.583333  0.583333         1.0  0.595745 NaN   

                                    
           min       50%       max  
test                                
0002  0.629167  0.629167  0.629167  
0003  0.811543  0.811543  0.811543  
0004  0.830549  0.830549  0.830549  
0005  0.930380  0.930380  0.930380  
0006  0.845133  0.845133  0.845133  
0007  0.704301  0.704301  0.704301  
0008  0.595745  0.595745  0.595745

by-event

In [26]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.687075,0.756219,0.629167
1,0003,0.844227,0.879442,0.811543
2,0004,0.764513,0.708502,0.830549
3,0005,0.629393,0.476726,0.930380
4,0006,0.644781,0.521739,0.845133
5,0007,0.697736,0.691293,0.704301
6,0008,0.589474,0.583333,0.595745


In [28]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           6.938855e-01
eventPrecision    6.596076e-01
eventRecall       7.638310e-01
dtype: float64

In [30]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16484\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.086722
eventPrecision    0.141024
eventRecall       0.123158
dtype: float64

by-sample

In [31]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.376633e-01
precision    6.725493e-01
recall       6.383141e-01
dtype: float64

In [32]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16484\634953098.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.080581
precision    0.149957
recall       0.130816
dtype: float64

In [33]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [34]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

### Restricted to N2

In [35]:
rawF1s=[]
rawPrecisions=[]
rawRecalls=[]

f1s=[]
precisions=[]
recalls=[]

eventF1s=[]
eventPrecisions=[]
eventRecalls=[]

checks=[]

for ind,row in experimentModels.iterrows():
    print('*************************')
    print(str(ind+1)+' of '+str(len(experimentModels)) )
    #load model
    model=loadDL(row.modelId,experimentId,datapath)
    testSubjectId=row.test
   
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Load features and labels
    #testFeatures=loadFeatureMatrix([testSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
    testLabels=loadLabelsVector([testSubjectId],usedAnnotations,signalsMetadata,samplerate)

    ##################################################
    hypnogram=loadCOGNITIONHypnogram(testSubjectId,cognipath)
    testLabels=testLabels*(hypnogram==2)
    ##################################################

    #Predict
    inferred=model.predict(paddedSignals[testSubjectId][np.newaxis,...,np.newaxis],verbose=0).flatten()
    rawLabels=inferred>=hyperThres

    ##################################################
    rawLabels=rawLabels*(hypnogram==2)
    ##################################################

    #Raw Metrics
    rawTp=np.sum(rawLabels*testLabels)
    rawFp=np.sum(rawLabels*(1-testLabels))
    rawTn=np.sum((1-rawLabels)*(1-testLabels))
    rawFn=np.sum((1-rawLabels)*testLabels)
    #Raw appends
    rawF1s.append(2*rawTp/(2*rawTp+rawFp+rawFn))
    rawPrecisions.append(rawTp/(rawTp+rawFp) )
    rawRecalls.append(rawTp/(rawTp+rawFn))
    #Process
    processedLabels=labelingProcess(rawLabels,hyperClose,hyperDuration,samplerate)
    #Processed metrics
    tp=np.sum(processedLabels*testLabels)
    fp=np.sum(processedLabels*(1-testLabels))
    tn=np.sum((1-processedLabels)*(1-testLabels))
    fn=np.sum((1-processedLabels)*testLabels)
    #Processed appends
    f1s.append(2*tp/(2*tp+fp+fn))
    precisions.append(tp/(tp+fp))
    recalls.append(tp/(tp+fn))

    #By-event metrics
    processedAnnotations=labelVectorToAnnotations(processedLabels,samplerate)
    processedAnnotations=processedAnnotations[processedAnnotations.duration<5]
    gtAnnotations=labelVectorToAnnotations(testLabels,samplerate)   #<- or just filter the annotations
    f,r,p=annotationPairToMetrics(gtAnnotations,processedAnnotations)
    
    #calculate metrics
    eventF1s.append(f)
    eventPrecisions.append(p)
    eventRecalls.append(r)

#include metrics in the dataframe
experimentModels['rawF1']=rawF1s
experimentModels['rawPrecision']=rawPrecisions
experimentModels['rawRecall']=rawRecalls

experimentModels['f1']=f1s
experimentModels['precision']=precisions
experimentModels['recall']=recalls

experimentModels['eventF1']=eventF1s
experimentModels['eventPrecision']=eventPrecisions
experimentModels['eventRecall']=eventRecalls


*************************
1 of 7
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-09-01 14:27:40         5064
metadata.json                                  2023-09-01 14:27:40           64
variables.h5                                   2023-09-01 14:27:40      1800912


Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\conv1d_2
......vars
.........0
.........1
...layers\conv1d_3
......vars
.........0
.........1
...layers\conv1d_4
......vars
.........0
.........1
...layers\conv1d_5
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
*************************
2 of 7
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    20

In [36]:
dumpPickle('experimentModelsTestN2_E1_temp.pkl',experimentModels)

In [37]:
experimentModels.columns

Index(['criteriumId', 'criteriumName', 'labelerIdList', 'train', 'val', 'test',
       'modelId', 'rawF1', 'rawPrecision', 'rawRecall', 'f1', 'precision',
       'recall', 'eventF1', 'eventPrecision', 'eventRecall'],
      dtype='object')

In [38]:
fig=px.scatter(experimentModels,x='rawF1',y='f1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [39]:
fig=px.scatter(experimentModels,x='rawF1',y='eventF1',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['rawF1'], y=experimentModels['rawF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [40]:
fig=px.scatter(experimentModels,x='eventF1',y='eventPrecision',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [41]:
fig=px.scatter(experimentModels,x='eventF1',y='eventRecall',color='test',hover_name='modelId', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()

In [42]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test').describe(percentiles=[0.5])

eventF1                                             eventPrecision  \
       count      mean std       min       50%       max          count   
test                                                                      
0002     1.0  0.724432 NaN  0.724432  0.724432  0.724432            1.0   
0003     1.0  0.851523 NaN  0.851523  0.851523  0.851523            1.0   
0004     1.0  0.793765 NaN  0.793765  0.793765  0.793765            1.0   
0005     1.0  0.636667 NaN  0.636667  0.636667  0.636667            1.0   
0006     1.0  0.678707 NaN  0.678707  0.678707  0.678707            1.0   
0007     1.0  0.719665 NaN  0.719665  0.719665  0.719665            1.0   
0008     1.0  0.620690 NaN  0.620690  0.620690  0.620690            1.0   

                                                 eventRecall                \
          mean std       min       50%       max       count      mean std   
test                                                                         
0002  0.805031 NaN  0.805031  0.805031  0.805031         1.0  0.658031 NaN   
0003  0.880890 NaN  0.880890  0.880890  0.880890         1.0  0.823892 NaN   
0004  0.746067 NaN  0.746067  0.746067  0.746067         1.0  0.848329 NaN   
0005  0.484848 NaN  0.484848  0.484848  0.484848         1.0  0.931373 NaN   
0006  0.570064 NaN  0.570064  0.570064  0.570064         1.0  0.839623 NaN   
0007  0.724719 NaN  0.724719  0.724719  0.724719         1.0  0.714681 NaN   
0008  0.627907 NaN  0.627907  0.627907  0.627907         1.0  0.613636 NaN   

                                    
           min       50%       max  
test                                
0002  0.658031  0.658031  0.658031  
0003  0.823892  0.823892  0.823892  
0004  0.848329  0.848329  0.848329  
0005  0.931373  0.931373  0.931373  
0006  0.839623  0.839623  0.839623  
0007  0.714681  0.714681  0.714681  
0008  0.613636  0.613636  0.613636

by-event

In [43]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean()

,test,eventF1,eventPrecision,eventRecall
0,0002,0.724432,0.805031,0.658031
1,0003,0.851523,0.880890,0.823892
2,0004,0.793765,0.746067,0.848329
3,0005,0.636667,0.484848,0.931373
4,0006,0.678707,0.570064,0.839623
5,0007,0.719665,0.724719,0.714681
6,0008,0.620690,0.627907,0.613636


In [44]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().mean()

test              2.857571e+23
eventF1           7.179212e-01
eventPrecision    6.913610e-01
eventRecall       7.756521e-01
dtype: float64

In [45]:
experimentModels[['test','eventF1','eventPrecision','eventRecall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16484\2931376690.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



eventF1           0.082907
eventPrecision    0.138043
eventRecall       0.115290
dtype: float64

by-sample

In [46]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().mean()

test         2.857571e+23
f1           6.569524e-01
precision    6.969860e-01
recall       6.510960e-01
dtype: float64

In [47]:
experimentModels[['test','f1','precision','recall']].groupby('test',as_index=False).mean().std()

C:\Users\Gurdiel\AppData\Local\Temp\ipykernel_16484\634953098.py:1: FutureWarning:

The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



f1           0.077096
precision    0.148542
recall       0.122966
dtype: float64

In [48]:
auxPrecision=pd.DataFrame({
    'metric':'event precision',
    'value':experimentModels.eventPrecision,
    'event F1':experimentModels.eventF1
})

auxRecall=pd.DataFrame({
    'metric':'event recall',
    'value':experimentModels.eventRecall,
    'event F1':experimentModels.eventF1
})
visualTradeoff=pd.concat((auxPrecision,auxRecall))

In [49]:
fig=px.scatter(visualTradeoff,x='event F1',y='value',color='metric', marginal_y="histogram")
fig.add_trace(
    go.Scatter(x=experimentModels['eventF1'], y=experimentModels['eventF1'], name="identity", mode='lines',fill="toself")
)
fig.show()